In [1]:
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request as req
import pandas as pd
import requests
from io import StringIO
import requests
import time
from selenium.webdriver import Chrome
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
import os
from naver_crawler import *
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import re
import parser
import datetime
from urllib.request import urlopen
import webbrowser
import xmltodict
import json
from ast import literal_eval

<ipython-input-1-e9ca380508bf>:23: DeprecationWarning: The parser module is deprecated and will be removed in future versions of Python
  import parser


In [2]:
# os.chdir('C:\\Users\\shic\\Desktop\\Crawling\\Desktop\\Project')
os.chdir('C:\\Users\\kuri9\\Desktop\\Crawling\\Desktop\\Project')
delay=3
browser = Chrome()
browser.implicitly_wait(delay)

In [12]:
url = 'https://www.kisrating.co.kr/ratingsStatistics/statics_spread.do'
res = requests.get(url)
page_soup = BeautifulSoup(res.text,'lxml')
table = page_soup.select_one('div.table_ty1')
# html -> 문자열
table_html = str(table)
# pandas의 read_html로 테이블 정보 읽기
table_df = pd.read_html(table_html)[0]
table_df.set_index('구분',inplace=True)
table_df

,3월,6월,9월,1년,1년6월,2년,3년,5년
구분,,,,,,,,
국고채,0.52,0.65,0.82,0.95,1.16,1.31,1.46,1.70
AAA,0.87,0.97,1.12,1.21,1.37,1.57,1.73,1.99
AA+,0.93,1.00,1.16,1.24,1.41,1.61,1.78,2.04
AA,0.97,1.04,1.20,1.28,1.44,1.64,1.82,2.10
AA-,0.99,1.06,1.22,1.31,1.47,1.67,1.85,2.21
A+,1.19,1.26,1.42,1.48,1.63,1.79,2.01,2.61
A,1.33,1.40,1.59,1.66,1.81,2.00,2.32,3.06
A-,1.55,1.65,1.85,1.93,2.11,2.36,2.78,3.67
BBB+,2.19,2.62,3.13,3.39,3.90,4.59,5.33,5.74


## 한국신용공시

In [3]:
# 창 띄우기
# 신금투 폴더 : os.chdir('C:\\Users\\shic\\Desktop\\Crawling\\Desktop\\Project')
os.chdir('C:\\Users\\kuri9\\Desktop\\Crawling\\Desktop\\Project')
delay=3
browser = Chrome()
browser.implicitly_wait(delay)
url = 'https://www.kisrating.co.kr/ratings/hot_disclosure.do'
browser.get(url)
browser.maximize_window()

In [7]:
# 2010 ~ 2021 date_list 만들기
import time
from datetime import date,timedelta
today= date.today()
date_lst=[]
for i in range(141,1000):
    date_lst.append((today - timedelta(days=30*i)).strftime("%Y.%m.%d"))
len(date_lst),date_lst[0],date_lst[-1]

(859, '2009.12.20', '1939.07.01')

In [8]:
table_DF1 = pd.DataFrame()
table_DF2 = pd.DataFrame()
table_DF3 = pd.DataFrame()
table_DF4 = pd.DataFrame()
table_DF5 = pd.DataFrame()
table_DF6 = pd.DataFrame()
table_DF7 = pd.DataFrame()
table_DF8 = pd.DataFrame()

In [ ]:
for i in range(len(date_lst)-1):
    start_date = browser.find_element_by_xpath('//*[@id="startDt"]')
    start_date.clear()
    start_date.send_keys(date_lst[i+1])
    end_date = browser.find_element_by_xpath('//*[@id="endDt"]')
    end_date.clear()
    end_date.send_keys(date_lst[i])
    src_btn = browser.find_element_by_xpath('//*[@id="btnSearch"]')
    src_btn.click()
    browser.implicitly_wait(10)
    
    # 수동으로 2010년 ~ 데이터 선택
    result = browser.page_source
    soup = BeautifulSoup(result, 'html.parser')
    
    table = soup.select('div.table_ty1')
    table_html = str(table)
    
    # 1. 회사채
    table_df1 = pd.read_html(table_html)[0]
    new_index=[]
    for i in range(len(table_df1.columns)):
        new_index.append(table_df1.columns[i][0])
    table_df1.columns = new_index
    table_df1.drop(['Unnamed: 0_level_0','Unnamed: 8_level_0'],axis=1,inplace=True)
    table_df1['Type']='회사채'
    table_DF1=pd.concat([table_DF1,table_df1],axis=0)
    print("table_DF1:",table_DF1.shape)
    
    # 2. 기업어음
    table_df2 = pd.read_html(table_html)[1]
    table_df2.drop('Unnamed: 0',axis=1,inplace=True)
    table_df2['Type']='기업어음'
    table_DF2=pd.concat([table_DF2,table_df2],axis=0)
    print("table_DF2:",table_DF2.shape)
    
    # 3. 전자단기사채
    table_df3 = pd.read_html(table_html)[2]
    table_df3.drop('Unnamed: 0',axis=1,inplace=True)
    table_df3['Type']='전자단기사채'
    table_DF3=pd.concat([table_DF3,table_df3],axis=0)
    print("table_DF3:",table_DF3.shape)
    
    # 4. Issuer Rating
    table_df4 = pd.read_html(table_html)[3]
    new_index=[]
    for i in range(len(table_df4.columns)):
        new_index.append(table_df4.columns[i][1])
    table_df4.columns = new_index
    table_df4.drop(['Unnamed: 0_level_1','Unnamed: 6_level_1'],axis=1,inplace=True)
    table_df4.columns=['회사명', '재무기준일', '평가종류', '직전등급', '직전Outlook', '현재등급', '현재Outlook', '기준통화', '평가일','리포트']
    table_df4['Type']='Issuer Rating'
    table_DF4=pd.concat([table_DF4,table_df4],axis=0)
    print("table_DF4:",table_DF4.shape)
    
    # 5. 보험금지급능력평가
    table_df5 = pd.read_html(table_html)[4]
    new_index=[]
    for i in range(len(table_df5.columns)):
        new_index.append(table_df5.columns[i][1])
    table_df5.columns = new_index
    table_df5.drop(['Unnamed: 0_level_1','Unnamed: 6_level_1'],axis=1,inplace=True)
    table_df5.columns=['회사명', '재무기준일', '평가종류', '직전등급', '직전Outlook', '현재등급', '현재Outlook', '기준통화', '평가일','리포트']
    table_df5['Type']='보험금지급능력평가'
    table_DF5=pd.concat([table_DF5,table_df5],axis=0)
    print("table_DF5:",table_DF5.shape)
    
    # 6. 자산유동화증권
    table_df6 = pd.read_html(table_html)[5]
    table_df6.drop('Unnamed: 0',axis=1,inplace=True)
    table_df6['Type']='자산유동화증권'
    table_DF6=pd.concat([table_DF6,table_df6],axis=0)
    print("table_DF6:",table_DF6.shape)
    
    # 7. 유동화익스포져
    table_df7 = pd.read_html(table_html)[6]
    table_df7.drop('Unnamed: 0',axis=1,inplace=True)
    table_df7['Type']='유동화익스포져'
    table_DF7=pd.concat([table_DF7,table_df7],axis=0)
    print("table_DF7:",table_DF7.shape)

    # 8. Covered Bond
    table_df8 = pd.read_html(table_html)[7]
    table_df8.drop('Unnamed: 0',axis=1,inplace=True)
    table_df8['Type']='Covered Bond'
    table_DF8=pd.concat([table_DF8,table_df8],axis=0)
    print("table_DF8:",table_DF8.shape)


table_DF1: (0, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)
table_DF6: (0, 11)
table_DF7: (0, 12)
table_DF8: (0, 12)
table_DF1: (2, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)
table_DF6: (1, 11)
table_DF7: (0, 12)
table_DF8: (0, 12)
table_DF1: (4, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)
table_DF6: (2, 11)
table_DF7: (0, 12)
table_DF8: (0, 12)
table_DF1: (4, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)
table_DF6: (2, 11)
table_DF7: (0, 12)
table_DF8: (0, 12)
table_DF1: (4, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)
table_DF6: (2, 11)
table_DF7: (0, 12)
table_DF8: (0, 12)
table_DF1: (4, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)
table_DF6: (2, 11)
table_DF7: (0, 12)
table_DF8: (0, 12)
table_DF1: (4, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)


table_DF1: (30, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)
table_DF6: (5, 11)
table_DF7: (0, 12)
table_DF8: (0, 12)
table_DF1: (32, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)
table_DF6: (5, 11)
table_DF7: (0, 12)
table_DF8: (0, 12)
table_DF1: (32, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)
table_DF6: (5, 11)
table_DF7: (0, 12)
table_DF8: (0, 12)
table_DF1: (32, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)
table_DF6: (5, 11)
table_DF7: (0, 12)
table_DF8: (0, 12)
table_DF1: (32, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)
table_DF6: (5, 11)
table_DF7: (0, 12)
table_DF8: (0, 12)
table_DF1: (32, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)
table_DF6: (5, 11)
table_DF7: (0, 12)
table_DF8: (0, 12)
table_DF1: (32, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (

table_DF1: (36, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)
table_DF6: (5, 11)
table_DF7: (0, 12)
table_DF8: (0, 12)
table_DF1: (36, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)
table_DF6: (5, 11)
table_DF7: (0, 12)
table_DF8: (0, 12)
table_DF1: (36, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)
table_DF6: (5, 11)
table_DF7: (0, 12)
table_DF8: (0, 12)
table_DF1: (36, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)
table_DF6: (5, 11)
table_DF7: (0, 12)
table_DF8: (0, 12)
table_DF1: (36, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)
table_DF6: (5, 11)
table_DF7: (0, 12)
table_DF8: (0, 12)
table_DF1: (36, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)
table_DF6: (5, 11)
table_DF7: (0, 12)
table_DF8: (0, 12)
table_DF1: (36, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (

table_DF1: (36, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)
table_DF6: (5, 11)
table_DF7: (0, 12)
table_DF8: (0, 12)
table_DF1: (36, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)
table_DF6: (5, 11)
table_DF7: (0, 12)
table_DF8: (0, 12)
table_DF1: (36, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)
table_DF6: (5, 11)
table_DF7: (0, 12)
table_DF8: (0, 12)
table_DF1: (36, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)
table_DF6: (5, 11)
table_DF7: (0, 12)
table_DF8: (0, 12)
table_DF1: (36, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)
table_DF6: (5, 11)
table_DF7: (0, 12)
table_DF8: (0, 12)
table_DF1: (36, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)
table_DF6: (5, 11)
table_DF7: (0, 12)
table_DF8: (0, 12)
table_DF1: (36, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (

table_DF1: (36, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)
table_DF6: (5, 11)
table_DF7: (0, 12)
table_DF8: (0, 12)
table_DF1: (36, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)
table_DF6: (5, 11)
table_DF7: (0, 12)
table_DF8: (0, 12)
table_DF1: (36, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)
table_DF6: (5, 11)
table_DF7: (0, 12)
table_DF8: (0, 12)
table_DF1: (36, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)
table_DF6: (5, 11)
table_DF7: (0, 12)
table_DF8: (0, 12)
table_DF1: (36, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)
table_DF6: (5, 11)
table_DF7: (0, 12)
table_DF8: (0, 12)
table_DF1: (36, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (0, 11)
table_DF6: (5, 11)
table_DF7: (0, 12)
table_DF8: (0, 12)
table_DF1: (36, 11)
table_DF2: (0, 9)
table_DF3: (0, 10)
table_DF4: (0, 11)
table_DF5: (

In [198]:
table_DF1.평가일.str.slice(0,4).value_counts()

2020    2838
2021    2328
2019    2222
2018    1450
2017    1108
2016     670
2015     524
2014     268
2013     210
2012     138
2011      60
2010      10
Name: 평가일, dtype: int64

In [205]:
table_DF1=table_DF1.drop_duplicates()
table_DF2=table_DF2.drop_duplicates()
table_DF3=table_DF3.drop_duplicates()
table_DF4=table_DF4.drop_duplicates()
table_DF5=table_DF5.drop_duplicates()
table_DF6=table_DF6.drop_duplicates()
table_DF7=table_DF7.drop_duplicates()
table_DF8=table_DF8.drop_duplicates()

In [209]:
print("table_DF1:",table_DF1.shape)
print("table_DF2:",table_DF2.shape)
print("table_DF3:",table_DF3.shape)
print("table_DF4:",table_DF4.shape)
print("table_DF5:",table_DF5.shape)
print("table_DF6:",table_DF6.shape)
print("table_DF7:",table_DF7.shape)
print("table_DF8:",table_DF8.shape)

table_DF1: (11620, 11)
table_DF2: (299, 9)
table_DF3: (114, 10)
table_DF4: (119, 11)
table_DF5: (25, 11)
table_DF6: (5216, 11)
table_DF7: (420, 12)
table_DF8: (16, 12)


In [212]:
os.chdir('C:\\Users\\kuri9\\Desktop\\Crawling\\Desktop\\Project\\1_Data')
table_DF1.to_csv('KOREA_CREDIT_1.csv',index=False)
table_DF2.to_csv('KOREA_CREDIT_2.csv',index=False)
table_DF3.to_csv('KOREA_CREDIT_3.csv',index=False)
table_DF4.to_csv('KOREA_CREDIT_4.csv',index=False)
table_DF5.to_csv('KOREA_CREDIT_5.csv',index=False)
table_DF6.to_csv('KOREA_CREDIT_6.csv',index=False)
table_DF7.to_csv('KOREA_CREDIT_7.csv',index=False)
table_DF8.to_csv('KOREA_CREDIT_8.csv',index=False)

In [213]:
os.getcwd()

'C:\\Users\\kuri9\\Desktop\\Crawling\\Desktop\\Project\\1_Data'